# The Magnificents

 Garrick Byrne, Dalibor Hajdek, Lacey Harris, Diana Hidalgo

# Connecting to the database server

In [1]:
import json
import datetime
import hashlib
import random
import pandas as pd

from datetime import date

code = hashlib.md5()

with open('cred.json') as f:
    creds = json.load(f)

connection_string = "mysql://{user}:{password}@{host}".format(**creds)

import pymysql 
    
# loading the faker module
from faker import Faker
from faker_vehicle import VehicleProvider

from random import seed
from random import random
from random import randint

faker = Faker('en_US')
faker.add_provider(VehicleProvider)

# preparing the cursor connector
# establish a database connection
conn = pymysql.connect(host=creds['host'], user=creds['user'], passwd=creds['password'], db = 'magnificents_final_project', autocommit=True)
cursor = conn.cursor(cursor=pymysql.cursors.Cursor)

In [2]:
%reload_ext sql

# Creating the Tables

Note, due to the use of foreign keys, the tables must be created in a certain order.

Creating the User Table

In [ ]:
cursor.execute("""
DROP TABLE IF EXISTS `user`;
""")

cursor.execute("""
CREATE TABLE `user` (
  `driver_license` varchar(11) NOT NULL,
  `name` varchar(45) NOT NULL,
  `sex` char(1) NOT NULL CHECK (`sex` IN ("M","F")),
  PRIMARY KEY (`driver_license`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8
""")


Creating the Car Table

In [ ]:
# Note! Fields that are intended to refer to foreign keys must be the same data type as the field in the other
# table! 
cursor.execute("""
DROP TABLE IF EXISTS `user`;
""")

cursor.execute("""
CREATE TABLE `car` (
  `vin` varchar(30) NOT NULL,
  `owner_dl` varchar(11) NOT NULL,
  `make` varchar(20) NOT NULL,
  `model` varchar(75) NOT NULL,
  `year` year(4) NOT NULL,
  `body_type` varchar(75) NOT NULL,
  `fuel_type` varchar(45) NOT NULL,
  `seats` tinyint(3) unsigned DEFAULT NULL,
  `trunk_size_cuft` decimal(5,2) unsigned DEFAULT NULL,
  `fuel_economy_mpg` decimal(5,2) unsigned DEFAULT NULL,
  PRIMARY KEY (`vin`),
  KEY `car_dl_fk` (`owner_dl`),
  CONSTRAINT `car_dl_fk` FOREIGN KEY (`owner_dl`) REFERENCES `user` (`driver_license`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8
""")

Creating the Driver Table 

In [ ]:
cursor.execute("""
DROP TABLE IF EXISTS `driver`;
""")

cursor.execute("""
CREATE TABLE `driver` (
  `driver_license` varchar(11) NOT NULL,
  `sex` char(1) NOT NULL CHECK (`sex` IN ("M","F")),
  `date_of_birth` date NOT NULL,
  PRIMARY KEY (`driver_license`),
  CONSTRAINT `driver_dl_fk` FOREIGN KEY (`driver_license`) REFERENCES `user`(`driver_license`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8
""")

Creating the Availability Table

In [ ]:
cursor.execute("""
DROP TABLE IF EXISTS `availability`;
""")

cursor.execute("""
CREATE TABLE `availability` (
  `date` date NOT NULL,
  `vin` varchar(30) NOT NULL,
  `daily_rate` decimal(5,2) DEFAULT NULL,
  `excess_mileage_rate` decimal(5,2) DEFAULT NULL,
  `free_miles` int(5) unsigned NOT NULL,
  PRIMARY KEY (`date`,`vin`),
  KEY `availability_FK` (`vin`),
  CONSTRAINT `availability_FK` FOREIGN KEY (`vin`) REFERENCES `car` (`vin`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8
""")

Creating the Reservation Table

In [ ]:
cursor.execute("""
DROP TABLE IF EXISTS `reservation`;
""")

cursor.execute("""
CREATE TABLE `reservation` (
  `date` date NOT NULL,
  `vin` varchar(30) NOT NULL COMMENT 'vin is foreign key\n',
  `driver_license` varchar(11) DEFAULT NULL COMMENT 'foreign key to driver database\n',
  PRIMARY KEY (`date`,`vin`),
  KEY `reserv_VIN_fk` (`vin`),
  KEY `reserv_driver_fk` (`driver_license`),
  CONSTRAINT `reserv_VIN_fk` FOREIGN KEY (`vin`) REFERENCES `car` (`vin`),
  CONSTRAINT `reserv_driver_fk` FOREIGN KEY (`driver_license`) REFERENCES `driver` (`driver_license`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8
""")



Creating the Trip Table

In [ ]:
## Trip is completing the whole reservation. 
## A reservation corresponds to a trip. 1:1 relationship between reservation and trip. 

cursor.execute("""
DROP TABLE IF EXISTS `trip`;
""")

cursor.execute("""
CREATE TABLE `trip` (
  `date` date NOT NULL,
  `driver_license` varchar(45) NOT NULL,
  `vin` varchar(30) NOT NULL,
  `miles` decimal(6,2) DEFAULT NULL,
  PRIMARY KEY (`date`,`vin`),
  KEY `trip_VIN_fk` (`vin`),
  KEY `trip_driver_fk` (`driver_license`),
  CONSTRAINT `trip_VIN_fk` FOREIGN KEY (`vin`) REFERENCES `reservation` (`vin`),
  CONSTRAINT `trip_date_fk` FOREIGN KEY (`date`) REFERENCES `reservation` (`date`),
  CONSTRAINT `trip_driver_fk` FOREIGN KEY (`driver_license`) REFERENCES `reservation` (`driver_license`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8
""")

# Populating the Tables

1. User the faker module to populate at least 500 users.

In [ ]:

# inserting 250 women
for i in range(250):
    cursor.execute("""
    INSERT INTO user (driver_license, name, sex) VALUES (%s, %s, %s)""",
                   (round(random()*1000000), faker.name_female(), "F")
                  )
    

# inserting 250 men
for i in range(250):
    cursor.execute("""
    INSERT INTO user (driver_license, name, sex) VALUES (%s, %s, %s)""",
                   (round(random()*1000000), faker.name_male(), "M")
                  )

2. Add 300 cars owned by some of the user. A user may own multiple cars. 
   You can use the faker_vehicle plugin of the faker module.

In [ ]:
#Inserting 30 cars with diesel fuel type into car table.


cursor.execute("""
SELECT driver_license FROM user
""")
user_info = cursor.fetchall()
user_info = random.sample(user_info,30)
car_df = pd.DataFrame(user_info, columns=['driver_license'])


for i in tqdm.tqdm(range(car_df.shape[0])):
    
    car_info = faker.vehicle_object()
    make = car_info['Make']
    model = car_info['Model']
    year = car_info['Year']
    body_type = car_info['Category']


cursor.execute("""INSERT INTO magnificents_final_project.car (vin, owner_dl, make, model, year, body_type, fuel_type, seats, trunk_size_cuft, fuel_economy_mpg)
VALUES(%s,%s,%s,%s,%s,%s,'diesel',%s,%s,%s)
""", ( ( ''.join(random.choice(string.ascii_uppercase + string.digits) for i in range(16))), 
      car_df['driver_license'].iat[i],
      make, model, year, body_type,
      random.choice([2, 4]),
      round(random.uniform(5.0, 30.0),2),
      round(random.uniform(10.0, 50.0),2)))



In [ ]:
#Inserting 270 cars with gasoline fuel type into car table.


cursor.execute("""
SELECT driver_license FROM user
""")
user_info = cursor.fetchall()
user_info = random.sample(user_info,270)
car_df = pd.DataFrame(user_info, columns=['driver_license'])


for i in tqdm.tqdm(range(car_df.shape[0])):
    
    car_info = faker.vehicle_object()
    make = car_info['Make']
    model = car_info['Model']
    year = car_info['Year']
    body_type = car_info['Category']

cursor.execute("""INSERT INTO magnificents_final_project.car (vin, owner_dl, make, model, year, body_type, fuel_type, seats, trunk_size_cuft, fuel_economy_mpg)
VALUES(%s,%s,%s,%s,%s,%s,'gasoline',%s,%s,%s)
""", ( ( ''.join(random.choice(string.ascii_uppercase + string.digits) for i in range(16))), 
      car_df['driver_license'].iat[i],
      make, model, year, body_type,
      random.choice([2, 4]),
      round(random.uniform(5.0, 30.0),2),
      round(random.uniform(10.0, 50.0),2)))


3. Designate some users as drivers and provide their drivers license information.

In [ ]:
cursor.execute("""
SELECT driver_license, sex FROM user;
""")

user_info = cursor.fetchall()

user_info = random.sample(user_info, 150)

driver_df = pd.DataFrame(user_info, columns=['dl_number', 'sex'])

for i in range(driver_df.shape[0]):
    cursor.execute("""
       INSERT INTO driver (driver_license, sex, date_of_birth) VALUES (%s, %s, %s)""",
               (driver_df['dl_number'].iat[i], driver_df['sex'].iat[i], faker.date_of_birth())
               )

4. Make some cars available for rental for some dates between Dec 1, 2021 and Jan 31, 2022. 
   Have at least 6000 car days available.

In [ ]:
#Insterting into Availability table
cursor.execute("""
SELECT vin FROM car
""")
car_info = cursor.fetchall()
availability_df = pd.DataFrame(car_info, columns=['vin'])

start_date = datetime.date(2021, 12, 1)
end_date = datetime.date(2022, 1, 31)
increment = datetime.timedelta(days=1)


while start_date <= end_date:
    for i in tqdm.tqdm(range(0,100)):
        cursor.execute("""INSERT INTO magnificents_final_project.availability 
        (date, vin, daily_rate, excess_mileage_rate,free_miles) VALUES (%s, %s, %s, %s,%s);""", 
                       (start_date, 
                        availability_df['vin'].iat[i],       
                        round(random.uniform(20.0, 80.0),2),
                        round(random.uniform(.35, .80),2),
                        round(random.uniform(50, 200))))
    start_date =start_date+increment


In [5]:
#Inserting into reservation table

In [ ]:
start_date = datetime.date(2021, 12, 1)
end_date = datetime.date(2022, 1, 31)
increment = datetime.timedelta(days = 4)

while start_date <= end_date:
    
    for i in tqdm.tqdm(range(0,2)):
    
       # getting cars that are available only on the day in question
        cursor.execute("""SELECT vin FROM availability WHERE date=%s""", start_date)
        car_info = cursor.fetchall()
        reservation_df_vin = pd.DataFrame(car_info, columns=['vin'])
        vin = reservation_df_vin['vin'].iat
    
        vin = vin[i]

    #getting all drivers licneses
        cursor.execute("""SELECT driver_license FROM driver""")
        driver_info = cursor.fetchall()
        reservation_df_dl = pd.DataFrame(driver_info, columns=['driver_license'])
        driver_license = reservation_df_dl['driver_license'].iat
    
        driver_license = driver_license[i]
    

        

    #for i in tqdm.tqdm(range(0,25)):
        reserve_car(start_date,end_date, i , driver_license)
        
        start_date = start_date+increment


# Functions

1. Write the function get_available_cars that returns the list of cars available for rental and not already reserved.

In [7]:
def get_available_cars():
    
    cursor.execute("""
        SELECT `date`, vin, make, model, year, body_type
        FROM magnificents_final_project.availability
        JOIN magnificents_final_project.car USING(vin)
        WHERE (`date`, vin) NOT IN (
            SELECT `date`, vin 	
            FROM magnificents_final_project.reservation 	
        )
        GROUP BY magnificents_final_project.availability.`vin`, magnificents_final_project.availability.`date`;
    """)

    result = pd.DataFrame(cursor.fetchall(), columns =['date', 'vin', 'make', 'model', 'year', 'body_type'])

    #("The following is a list of cars that are available but not yet reserved.")

    return(result)

In [8]:
# testing the function
get_available_cars()


,date,vin,make,model,year,body_type
0,2021-12-01,06B6Z2I2DE3T779Y,Ford,Five Hundred,2005,Sedan
1,2021-12-07,06B6Z2I2DE3T779Y,Ford,Five Hundred,2005,Sedan
2,2021-12-08,06B6Z2I2DE3T779Y,Ford,Five Hundred,2005,Sedan
3,2021-12-09,06B6Z2I2DE3T779Y,Ford,Five Hundred,2005,Sedan
4,2021-12-10,06B6Z2I2DE3T779Y,Ford,Five Hundred,2005,Sedan
...,...,...,...,...,...,...
5872,2022-01-27,ZUVDB6K1BEOFTFHW,Kia,Sephia,1998,Sedan
5873,2022-01-28,ZUVDB6K1BEOFTFHW,Kia,Sephia,1998,Sedan
5874,2022-01-29,ZUVDB6K1BEOFTFHW,Kia,Sephia,1998,Sedan
5875,2022-01-30,ZUVDB6K1BEOFTFHW,Kia,Sephia,1998,Sedan


2. Write the function reserve_car that creates a reservation with a start date and an end date.

In [ ]:
def reserve_car(start_date, end_date, vin, driver_license):
    delta = datetime.timedelta(days=1)
    while start_date <= end_date:
        try:
            cursor.execute("""INSERT INTO reservation (date, vin, driver_license) VALUES (%s,%s, %s)""",
                        (start_date, vin, driver_license))
        except:
            print("This given car is not available during those dates.\nPlease create a new reservation.")
            break
        else:
            print("A reservation has been created.")
        start_date=start_date+delta


TESTING ROUTINES

In [ ]:
# This function call should throw an error because the vehicle in question is already reserved 
# for one (or more) of the days. 

start_date = datetime.date(2021, 12, 1)
end_date = datetime.date(2021, 12, 4)
vin = '0838SWLG31495EAB'
driver_license = '10381'
reserve_car(start_date, end_date, vin, driver_license)

In [ ]:
# This function call should throw an error because the driver_license number is not in the 
# drivers table.

start_date = datetime.date(2021, 12, 8)
end_date = datetime.date(2021, 12, 9)
vin = '0838SWLG31495EAB'
driver_license = '11465'

reserve_car(start_date, end_date, vin, driver_license)

In [ ]:
# This function call should throw an error because the VIN is not in the 
# availability table.

start_date = datetime.date(2021, 12, 8)
end_date = datetime.date(2021, 12, 9)
vin = 'blah'
driver_license = '10381'

reserve_car(start_date, end_date, vin, driver_license)

In [ ]:
# This function call should run successfully, at least the first time!

start_date = datetime.date(2021, 12, 1)
end_date = datetime.date(2021, 12, 4)
vin = '099PJFX2XYP0YIGJ'
driver_license = '10381'

reserve_car(start_date, end_date, vin, driver_license)

3. Write the function start_trip and complete_trip

In [ ]:
def start_trip(date, vin, driver_license):
    conn.commit()
    
    try: 
        cursor.execute("""INSERT INTO trip (date, vin, driver_license) VALUES (%s,%s, %s)""",
                  (date, vin, driver_license))
    except pymysql.IntegrityError as error:
        code, message = error.args
        print("There is something anomalous about this request, specifically:")
        print (">>>", code, message)
        print("Please make the necessary adjustments and try again.")
        print("The function will now terminate.")
        
    else: 
        print("The request was processed successfully!")

In [ ]:
# populating start the trip table using start_trips()
cursor.execute("""
        SELECT *  
        FROM magnificents_final_project.reservation;
    """)

result = pd.DataFrame(cursor.fetchall(), columns =['date', 'vin', 'driver_license'])

for index, row in result.iterrows():
    
    start_trip(row['date'], row['vin'], row['driver_license'])

In [ ]:
def complete_trip(date, vin, miles):
    conn.commit()
    
    try: 
        cursor.execute("""UPDATE trip SET miles =%s WHERE date=%s AND vin = %s""",
                  (miles, date, vin))
    except pymysql.IntegrityError as error:
        code, message = error.args
        print("There is something anomalous about this request, specifically:")
        print (">>>", code, message)
        print("Please make the necessary adjustments and try again.")
        print("The function will now terminate.")
        
    else: 
        print("The request was processed successfully!")

In [ ]:
# updating the trip table to include information about mileage. the trips table using start_trips()
cursor.execute("""
        SELECT date, vin
        FROM magnificents_final_project.trip;
    """)

result = pd.DataFrame(cursor.fetchall(), columns =['date', 'vin'])

for index, row in result.iterrows():
    
    complete_trip(row['date'], row['vin'], randint(20,125))

# Queries

1. List all drivers who have reserved a car between Christmas and New Years.

In [ ]:
cursor.execute('''SELECT user.name,
                         driver.driver_license,
                         reservation.date
                         FROM magnificents_final_project.user
                         INNER JOIN magnificents_final_project.driver
                         USING (driver_license) 
                         INNER JOIN magnificents_final_project.reservation
                         USING (driver_license)
                         WHERE DATE(reservation.date) >= '2021-12-25' 
                         AND DATE(reservation.date) <= '2022-1-2'
                         GROUP BY user.name''')


2. Pick one car and show all the drivers who have rented it.

In [10]:
cursor.execute('''SELECT car.make,
                         car.model,
                         reservation.date,
                         driver.driver_license,
                         reservation.vin,
                         user.name
                         FROM magnificents_final_project.car
                         INNER JOIN magnificents_final_project.reservation
                         USING (vin)
                         INNER JOIN magnificents_final_project.driver
                         USING (driver_license) 
                         INNER JOIN magnificents_final_project.user
                         USING (driver_license)
                         GROUP BY reservation.vin
                         LIMIT 1''')


1

In [11]:
cursor.fetchall()

(('Ford',
  'Five Hundred',
  datetime.date(2021, 12, 2),
  '108229',
  '06B6Z2I2DE3T779Y',
  'Belinda Murray'),)

3. List all drivers who have rented a car before they turned 25 years old.

In [ ]:
cursor.execute('''SELECT user.name,
                         driver.date_of_birth,
                         reservation.date
                         FROM magnificents_final_project.user
                         INNER JOIN magnificents_final_project.driver
                         USING (driver_license) 
                         INNER JOIN magnificents_final_project.reservation
                         USING (driver_license)
                         WHERE (ROUND (DATEDIFF(NOW(), driver.date_of_birth)/365)) < 25
                         GROUP BY user.name''')


4. List all male drivers aged 65 and over.

In [ ]:
from dateutil.relativedelta import relativedelta

# The folks born before the date calculated below are >= 65 years of age
# on the date the query is being run!
cutoff_date = date.today()-relativedelta(years=65) 

cursor.execute("""
SELECT driver.driver_license, name, driver.sex, date_of_birth 
FROM driver
JOIN `user` USING(driver_license)
WHERE date_of_birth < %s AND driver.sex = "M";""", cutoff_date) 

result = pd.DataFrame(cursor.fetchall(), columns =['driver license', 'name', 'sex', 'date_of_birth'])

print("The following is a list of male drivers who are >= 65 years of age.")

print(result)

5. List the total number of cars available and not yet reserved for each night 
   at a price under $50 and seating at least four people.

In [ ]:
cursor.execute("""
SELECT COUNT(availability.vin) AS number_of_vehicles_available, availability.`date` 
FROM availability 
JOIN car USING(vin)
WHERE (availability.vin, availability.`date`) NOT IN(
SELECT reservation.vin, reservation.date 
FROM reservation ) AND seats >= 4 AND daily_rate < 50
GROUP BY availability.`date`
""")
result = pd.DataFrame(cursor.fetchall(), columns =['number of vehicles available', 'date'])

print("The following is a list dates and how many vehicles are available on each date.")

print(result)


6. List all completed trips and the amount owed based on the mileage logged and the prices provided.

In [12]:
cursor.execute("""
select excess_mileage_calc.date, excess_mileage_calc.vin, excess_miles, excess_mileage_charge, daily_rate, (excess_mileage_charge+ daily_rate) AS total from 
(SELECT trip.date, trip.vin, GREATEST((availability.free_miles-trip.miles), 0) AS excess_miles
FROM trip
JOIN availability ON trip.`date` = availability.`date` AND trip.vin = availability.vin) as excess_mileage_calc
JOIN
(SELECT trip.date, trip.vin, GREATEST(availability.free_miles-trip.miles, 0) * availability.excess_mileage_rate AS excess_mileage_charge
FROM trip
JOIN availability ON trip.`date` = availability.`date` AND trip.vin = availability.vin) AS excess_mileage_charge_calc
ON excess_mileage_charge_calc.date = excess_mileage_calc.date AND excess_mileage_charge_calc.vin = excess_mileage_calc.vin
JOIN availability ON excess_mileage_calc.date =availability.date AND excess_mileage_calc.vin = availability.vin""")

result = pd.DataFrame(cursor.fetchall(), columns =['date', 'vin', 'excess miles', 'excess mileage charge',
                                                  'daily rate', 'total charges due'])
(result)

,date,vin,excess miles,excess mileage charge,daily rate,total charges due
0,2021-12-01,9FEYV4YWCPNO9U0X,0.00,0.0000,28.73,28.7300
1,2021-12-02,06B6Z2I2DE3T779Y,0.00,0.0000,22.80,22.8000
2,2021-12-02,9FEYV4YWCPNO9U0X,102.00,61.2000,60.95,122.1500
3,2021-12-02,9NLDS94LF4703C31,81.00,38.8800,77.95,116.8300
4,2021-12-02,IC1R2CB61Q8K1AKV,98.00,44.1000,39.00,83.1000
...,...,...,...,...,...,...
318,2021-12-30,9DIZ5X2U4B1Y8FYL,0.00,0.0000,43.61,43.6100
319,2021-12-30,HWFNDXHFLY545I7O,51.00,33.6600,68.92,102.5800
320,2021-12-31,99T87UQCJJ8YTUJS,9.00,4.4100,69.76,74.1700
321,2021-12-31,9DIZ5X2U4B1Y8FYL,69.00,28.2900,39.36,67.6500


In [ ]:
7. Show the car makes sorted by the number of reservations.

In [ ]:
cursor.execute('''SELECT car.make,
                         COUNT(reservation.vin) AS num_of_reservation
                         FROM magnificents_final_project.car
                         INNER JOIN magnificents_final_project.reservation
                         USING (vin)
                         GROUP BY car.make
                         ORDER BY num_of_reservation DESC ''')


8. For each driver, show the total miles driven in their completed trips.

In [15]:
cursor.execute('''SELECT user.name,
                         trip.driver_license,
                         SUM(trip.miles)
                         FROM magnificents_final_project.user
                         INNER JOIN magnificents_final_project.driver
                         USING (driver_license)
                         INNER JOIN magnificents_final_project.trip
                         USING (driver_license)
                         GROUP BY trip.driver_license
                         ORDER BY user.name''')


63

In [16]:
cursor.fetchall()

(('Adam Hinton', '137485', Decimal('300.00')),
 ('Alexandra Frazier', '223141', Decimal('526.00')),
 ('Alicia Pope', '670207', Decimal('319.00')),
 ('Alyssa Garza DVM', '25799', Decimal('313.00')),
 ('Amy Walker', '465541', Decimal('456.00')),
 ('Belinda Murray', '108229', Decimal('472.00')),
 ('Benjamin Franco', '520954', Decimal('465.00')),
 ('Beth Moreno', '524315', Decimal('678.00')),
 ('Brandon Hughes', '109250', Decimal('353.00')),
 ('Christine Caldwell', '166157', Decimal('290.00')),
 ('Christine Kemp', '230467', Decimal('399.00')),
 ('Christopher Marshall', '512475', Decimal('656.00')),
 ('Danny Beard', '167631', Decimal('416.00')),
 ('Dawn Lee', '154408', Decimal('284.00')),
 ('Deborah Santos', '506414', Decimal('606.00')),
 ('Denise Robinson', '161254', Decimal('395.00')),
 ('Diane Compton', '238410', Decimal('462.00')),
 ('Elizabeth Jackson', '437092', Decimal('413.00')),
 ('Elizabeth Lopez', '149441', Decimal('249.00')),
 ('Heather Simpson', '685934', Decimal('340.00')),
 (